In [2]:
from datetime import datetime
import pandas as pd
import pygwalker as pyg
import matplotlib.pyplot as plt

In [203]:

#import dataset
data_type = {
    "_CustomerID": str,
    "_SalesTeamID":str,
    "_ProductID":str,
    "_StoreID":str
}

df = pd.read_csv("sales_data.csv",dtype= data_type,parse_dates=["OrderDate","ProcuredDate","ShipDate","DeliveryDate"])


FileNotFoundError: [Errno 2] No such file or directory: 'rfm.csv'

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   OrderNumber       7991 non-null   object        
 1   Sales Channel     7991 non-null   object        
 2   WarehouseCode     7991 non-null   object        
 3   ProcuredDate      7991 non-null   datetime64[ns]
 4   OrderDate         7991 non-null   datetime64[ns]
 5   ShipDate          7991 non-null   datetime64[ns]
 6   DeliveryDate      7991 non-null   datetime64[ns]
 7   CurrencyCode      7991 non-null   object        
 8   _SalesTeamID      7991 non-null   object        
 9   _CustomerID       7991 non-null   object        
 10  _StoreID          7991 non-null   object        
 11  _ProductID        7991 non-null   object        
 12  Order Quantity    7991 non-null   int64         
 13  Discount Applied  7991 non-null   float64       
 14  Unit Price        7991 n

In [169]:
start_date = df["OrderDate"].min()
end_date = start_date + pd.DateOffset(years=1)

# Filter data for the first year of business
first_year_df = df[(df["OrderDate"] >= start_date) & (df["OrderDate"] <= end_date)]


In [170]:
# To create new dataframe called sales_df
sales_df = df.copy()

In [171]:
# To calculate Revenue, Unit Price after discount rate -Cost * Quantity

sales_df["Revenue"] = round((df["Unit Price"] - (df["Unit Price"]*  df["Discount Applied"])-df["Unit Cost"]) \
                            *  df["Order Quantity"],2)

In [10]:
#creating first 6month
first_6m_df = sales_df.set_index("OrderDate").first("6M").reset_index()


In [11]:
today_date = pd.to_datetime("2021-01-01")

In [172]:
sales_df.columns

Index(['OrderNumber', 'Sales Channel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', '_SalesTeamID',
       '_CustomerID', '_StoreID', '_ProductID', 'Order Quantity',
       'Discount Applied', 'Unit Price', 'Unit Cost', 'Revenue'],
      dtype='object')

In [173]:
rfm_cols = ["OrderNumber","_CustomerID","OrderDate","Revenue"]

df1 = sales_df[rfm_cols]

df1

,OrderNumber,_CustomerID,OrderDate,Revenue
0,SO - 000101,15,2018-05-31,4073.43
1,SO - 000102,20,2018-05-31,886.41
2,SO - 000103,16,2018-05-31,905.50
3,SO - 000104,48,2018-05-31,5486.76
4,SO - 000105,49,2018-05-31,1312.13
...,...,...,...,...
7986,SO - 0008087,41,2020-12-30,94.97
7987,SO - 0008088,29,2020-12-30,6725.46
7988,SO - 0008089,32,2020-12-30,1339.00
7989,SO - 0008090,42,2020-12-30,1286.40


In [176]:
rfm_dataset =df1.groupby("_CustomerID").agg({
    'OrderDate': lambda x : (today_date - x.max()).days,
    "OrderNumber": 'count',
    'Revenue': 'sum'
      
})

rfm_dataset.head(2)

,OrderDate,OrderNumber,Revenue
_CustomerID,,,
1,9,152,335933.63
10,15,158,435122.22


In [177]:
rfm_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 1 to 9
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   OrderDate    50 non-null     int64  
 1   OrderNumber  50 non-null     int64  
 2   Revenue      50 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.6+ KB


In [180]:
# change column name
rfm_dataset.rename(
    columns = {
        "OrderDate":"recency",
        "OrderNumber":"frequency",
        "Revenue":"monetary"
    },inplace=True

)




In [181]:
#Assign 1-5 score based on their Recency , Frequency, and Monetary by qcut method
r = pd.qcut(rfm_dataset["recency"],q=5,labels=range(5,0,-1))
f = pd.qcut(rfm_dataset["frequency"],q=5,labels=range(1,6))
m = pd.qcut(rfm_dataset["monetary"],q=5,labels=range(1,6))


In [182]:
# Assign scores to R, F, and M columns
rfm = rfm_dataset.assign(R=r.values,F=f.values,M=m.values)

In [187]:
rfm["rfm_group"] = rfm[["R","F","M"]].apply(lambda x: "".join(x.astype(str)),axis=1)

In [188]:
rfm["rfm_total"] = rfm[["R","F","M"]].sum(axis=1)

In [189]:
rfm

,recency,frequency,monetary,R,F,M,rfm_group,rfm_total
_CustomerID,,,,,,,,
1,9,152,335933.63,2,2,1,221,5
10,15,158,435122.22,1,3,3,133,7
11,6,178,487614.24,3,5,5,355,13
12,3,210,616719.31,5,5,5,555,15
13,4,171,441003.32,4,4,3,443,11
14,5,157,381450.02,3,3,2,332,8
15,4,142,441668.35,4,1,4,414,9
16,3,135,402938.75,5,1,2,512,8
17,6,175,534027.38,3,5,5,355,13


In [196]:
seg_map = {
    r'[1-2][1-3][1-4]': 'Need Attention',
    r'[1-2][2-5][3-5]': 'Cant Lose Group',
    r'[2-4][2-4][1-3]': 'At Risk Customer',
    r'[3-5][1-2][1]': 'Checker',
    r'[5][1][1]': 'New Customers',
    r'[3-5][1-5][2-4]': 'Potential Loyalists',
    r'[3-5][2-5][4-5]': 'Champions'
}

rfm["segment"] = rfm["rfm_group"].replace(seg_map,regex=True)
rfm["segment"].value_counts()

Potential Loyalists    13
Need Attention         12
Cant Lose Group         8
Champions               7
At Risk Customer        7
Checker                 3
Name: segment, dtype: int64

In [197]:
rfm

,recency,frequency,monetary,R,F,M,rfm_group,rfm_total,segment
_CustomerID,,,,,,,,,
1,9,152,335933.63,2,2,1,221,5,Need Attention
10,15,158,435122.22,1,3,3,133,7,Need Attention
11,6,178,487614.24,3,5,5,355,13,Champions
12,3,210,616719.31,5,5,5,555,15,Champions
13,4,171,441003.32,4,4,3,443,11,At Risk Customer
14,5,157,381450.02,3,3,2,332,8,At Risk Customer
15,4,142,441668.35,4,1,4,414,9,Potential Loyalists
16,3,135,402938.75,5,1,2,512,8,Potential Loyalists
17,6,175,534027.38,3,5,5,355,13,Champions


In [199]:
sales_df1 = sales_df.drop(columns=["OrderNumber","WarehouseCode","ProcuredDate","ShipDate","DeliveryDate","CurrencyCode","Order Quantity","Unit Price","Unit Cost"])

In [204]:
final_df = sales_df1.merge(rfm,how="left",on="_CustomerID")
final_df.to_csv("rfm.csv",index=False)

,Sales Channel,OrderDate,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Discount Applied,Revenue,recency,frequency,monetary,R,F,M,rfm_group,rfm_total,segment
0,In-Store,2018-05-31,6,15,259,12,0.075,4073.43,4,142,441668.35,4,1,4,414,9,Potential Loyalists
1,Online,2018-05-31,14,20,196,27,0.075,886.41,9,167,439147.97,2,4,3,243,9,Cant Lose Group
2,Distributor,2018-05-31,21,16,213,16,0.050,905.50,3,135,402938.75,5,1,2,512,8,Potential Loyalists
3,Wholesale,2018-05-31,28,48,107,23,0.075,5486.76,10,172,424970.73,2,5,3,253,10,Cant Lose Group
4,Distributor,2018-05-31,22,49,111,26,0.100,1312.13,7,152,354161.33,2,2,1,221,5,Need Attention
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7986,In-Store,2020-12-30,9,41,339,29,0.075,94.97,2,161,403693.28,5,3,2,532,10,Potential Loyalists
7987,Online,2020-12-30,14,29,202,3,0.050,6725.46,2,179,531770.74,5,5,5,555,15,Champions
7988,Online,2020-12-30,14,32,241,35,0.200,1339.00,2,173,435206.07,5,5,3,553,13,Potential Loyalists
7989,Online,2020-12-30,20,42,112,36,0.100,1286.40,2,161,427159.63,5,3,3,533,11,Potential Loyalists
